In [1]:
from PIL import Image # used for loading images
import numpy as np
import os # used for navigating to image path
import matplotlib.pyplot as plt
import cv2
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, MaxPooling2D, Conv2D
from keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization, regularizers
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

Using TensorFlow backend.


https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

In [5]:
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest')


In [6]:
batch_size = 16

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest')

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        'data/train',  # this is the target directory
        target_size=(300, 300)
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        'data/validation',
        target_size=(300, 300),
        batch_size=batch_size,
        class_mode='binary')

Found 4927 images belonging to 2 classes.
Found 542 images belonging to 2 classes.


In [7]:
model = Sequential()
model.add(Conv2D(32, kernel_size = (3, 3), activation='relu', input_shape=(300, 300, 3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Conv2D(96, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Conv2D(32, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [8]:
history = model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=800 // batch_size,
        callbacks=callbacks_list)

model.save("model2-added-dropout.h5")


Epoch 1/50
125/125 [==============================] - 1022s 8s/step - loss: 0.6046 - acc: 0.7790 - val_loss: 0.5757 - val_acc: 0.8221
Epoch 2/50
125/125 [==============================] - 919s 7s/step - loss: 0.4425 - acc: 0.8325 - val_loss: 0.7910 - val_acc: 0.8346
Epoch 3/50
125/125 [==============================] - 921s 7s/step - loss: 0.3649 - acc: 0.8584 - val_loss: 0.6619 - val_acc: 0.8668
Epoch 4/50
125/125 [==============================] - 933s 7s/step - loss: 0.3273 - acc: 0.8690 - val_loss: 1.3092 - val_acc: 0.6353
Epoch 5/50
125/125 [==============================] - 939s 8s/step - loss: 0.2921 - acc: 0.8814 - val_loss: 0.2749 - val_acc: 0.9083
Epoch 6/50
125/125 [==============================] - 945s 8s/step - loss: 0.2908 - acc: 0.8900 - val_loss: 0.4708 - val_acc: 0.7769
Epoch 7/50
125/125 [==============================] - 951s 8s/step - loss: 0.2638 - acc: 0.8975 - val_loss: 0.3487 - val_acc: 0.8957
Epoch 8/50
125/125 [==============================] - 943s 8s/step -